In [1]:
#Update link
!wget 'https://raw.githubusercontent.com/visheshks04/speech-recognition/master/data/captions_data.json?token=ALKS45TCFDUJ5Z6YUTAFPVDBO3UUU'

--2021-10-25 17:28:04--  https://raw.githubusercontent.com/visheshks04/speech-recognition/master/data/captions_data.json?token=ALKS45TCFDUJ5Z6YUTAFPVDBO3UUU
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 132177 (129K) [text/plain]
Saving to: ‘captions_data.json?token=ALKS45TCFDUJ5Z6YUTAFPVDBO3UUU’

captions_data.json? 100%[===================>] 129.08K  --.-KB/s    in 0.02s   

2021-10-25 17:28:04 (5.60 MB/s) - ‘captions_data.json?token=ALKS45TCFDUJ5Z6YUTAFPVDBO3UUU’ saved [132177/132177]



In [3]:
#Update filename here
!mv captions_data.json?token=ALKS45TCFDUJ5Z6YUTAFPVDBO3UUU captions_data.json

## Requirements

In [27]:
!pip3 install pytube

In [28]:
!pip3 install torchaudio

In [29]:
!pip3 install git+https://github.com/PyTorchLightning/pytorch-lightning fsspec --no-deps --target=$nb_path

  Cloning https://github.com/PyTorchLightning/pytorch-lightning to /tmp/pip-req-build-f_we_o_k
  Running command git clone -q https://github.com/PyTorchLightning/pytorch-lightning /tmp/pip-req-build-f_we_o_k
  Running command git submodule update --init --recursive -q
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done


## Fetch Video

In [12]:
from pytube import YouTube

url = 'https://youtu.be/HtSuA80QTyo'

yt = YouTube(url)
print(f'Downloading {yt.title}')
audio = yt.streams.get_by_itag(251)
audio.download('.')

'/content/./1 Algorithmic Thinking Peak Finding.webm'

# Convert to wav

In [13]:
import librosa
from scipy.io import wavfile

aud = librosa.load('1 Algorithmic Thinking Peak Finding.webm')
wavfile.write('wavAudio.wav', aud[1], aud[0])

/usr/local/lib/python3.7/dist-packages/librosa/core/audio.py:165: UserWarning: PySoundFile failed. Trying audioread instead.
  warnings.warn("PySoundFile failed. Trying audioread instead.")


## Creating JSON

In [14]:
import json

data = []

with open('captions_data.json', 'r') as f:
    captions_data = json.load(f)

for event in captions_data['events']:
    sample = dict()
    sample["key"] = str(event['tStartMs']) + '.wav'
    sample["text"] = event['segs'][0]['utf8']
    data.append(sample)


with open('dataset.json', 'w') as f:
    json.dump(data, f, indent=4)

## Chopping clips

In [15]:
!mkdir clips

In [16]:
with open('captions_data.json', 'r') as f:
    data = json.load(f)

aud = librosa.load('wavAudio.wav')

for event in data['events']:
    start_index = event['tStartMs'] * 1e-3 * aud[1]
    end_index = (event['tStartMs'] + event['dDurationMs']) * 1e-3 * aud[1]

    start_index, end_index = int(start_index), int(end_index)

    chopped_sample = aud[0][start_index:end_index]
    wavfile.write('clips/{}.wav'.format(event['tStartMs']), aud[1], chopped_sample)

## Cleaning Dataset.json

In [17]:
def clean_data(dataset_path):

    with open(dataset_path, 'r') as f:
        data = json.load(f)

    for event in data:
        sentence = event['text']
        sentence = sentence.replace('\n', ' ')
        sentence = remove_caps(sentence)
        sentence = remove_special_chars(sentence)
        sentence = sentence.lower()
        event['text'] = sentence

    with open(dataset_path, 'w') as f:
        json.dump(data, f, indent=4)


def remove_caps(string):
    string = string.split()
    for word in string:
        if(word.isupper()):
            string.remove(word)
    string = " ".join(string)
    return string

def remove_special_chars(string):

    removables = ['.', ',', ';', '"', '\'', ':', '<', '>', '?', '/', '\\', '[', ']', '{', '}', '-', '_', '+', '=', '(', ')', '!', '@', '#', '$', '%', '^', '&', '*', '~', '`']

    string = list(string)

    for ch in string:
        if ch in removables:
            string.remove(ch)

    string = "".join(string)
    return string

In [18]:
clean_data('dataset.json')

## Train-Dev-Test Split

In [25]:
def train_dev_test(file_path, train_size = 0.8, dev_size = 0.1):
  with open(file_path, 'r') as f:
    ds = json.load(f)

  n = len(ds)

  train_ind = int(n*train_size)
  dev_ind = int(train_ind + n*dev_size)

  train_set = ds[:train_ind]
  dev_set = ds[train_ind:dev_ind]
  test_set = ds[dev_ind:]

  with open('train.json', 'w') as f:
    json.dump(train_set, f, indent = 4)

  with open('dev.json', 'w') as f:
    json.dump(dev_set, f, indent = 4)

  with open('test.json', 'w') as f:
    json.dump(test_set, f, indent = 4)


train_dev_test('dataset.json')

In [20]:
import torch

class TextProcess:
	def __init__(self):
		char_map_str = """
		' 0
		<SPACE> 1
		a 2
		b 3
		c 4
		d 5
		e 6
		f 7
		g 8
		h 9
		i 10
		j 11
		k 12
		l 13
		m 14
		n 15
		o 16
		p 17
		q 18
		r 19
		s 20
		t 21
		u 22
		v 23
		w 24
		x 25
		y 26
		z 27
		"""
		self.char_map = {}
		self.index_map = {}
		for line in char_map_str.strip().split('\n'):
			ch, index = line.split()
			self.char_map[ch] = int(index)
			self.index_map[int(index)] = ch
		self.index_map[1] = ' '

	def text_to_int_sequence(self, text):
		""" Use a character map and convert text to an integer sequence """
		int_sequence = []
		for c in text:
			if c == ' ':
				ch = self.char_map['<SPACE>']
			else:
				ch = self.char_map[c]
			int_sequence.append(ch)
		return int_sequence

	def int_to_text_sequence(self, labels):
		""" Use a character map and convert integer labels to an text sequence """
		string = []
		for i in labels:
			string.append(self.index_map[i])
		return ''.join(string).replace('<SPACE>', ' ')


textprocess = TextProcess()

def GreedyDecoder(output, labels, label_lengths, blank_label=28, collapse_repeated=True):
	arg_maxes = torch.argmax(output, dim=2)
	decodes = []
	targets = []
	for i, args in enumerate(arg_maxes):
		decode = []
		targets.append(textprocess.int_to_text_sequence(
				labels[i][:label_lengths[i]].tolist()))
		for j, index in enumerate(args):
			if index != blank_label:
				if collapse_repeated and j != 0 and index == args[j -1]:
					continue
				decode.append(index.item())
		decodes.append(textprocess.int_to_text_sequence(decode))
	return decodes, targets

In [21]:
import torch
import torchaudio
import torch.nn as nn
import pandas as pd
import numpy as np

# NOTE: add time stretch
class SpecAugment(nn.Module):

    def __init__(self, rate, policy=3, freq_mask=15, time_mask=35):
        super(SpecAugment, self).__init__()

        self.rate = rate

        self.specaug = nn.Sequential(
            torchaudio.transforms.FrequencyMasking(freq_mask_param=freq_mask),
            torchaudio.transforms.TimeMasking(time_mask_param=time_mask)
        )

        self.specaug2 = nn.Sequential(
            torchaudio.transforms.FrequencyMasking(freq_mask_param=freq_mask),
            torchaudio.transforms.TimeMasking(time_mask_param=time_mask),
            torchaudio.transforms.FrequencyMasking(freq_mask_param=freq_mask),
            torchaudio.transforms.TimeMasking(time_mask_param=time_mask)
        )

        policies = { 1: self.policy1, 2: self.policy2, 3: self.policy3 }
        self._forward = policies[policy]

    def forward(self, x):
        return self._forward(x)

    def policy1(self, x):
        probability = torch.rand(1, 1).item()
        if self.rate > probability:
            return  self.specaug(x)
        return x

    def policy2(self, x):
        probability = torch.rand(1, 1).item()
        if self.rate > probability:
            return  self.specaug2(x)
        return x

    def policy3(self, x):
        probability = torch.rand(1, 1).item()
        if probability > 0.5:
            return self.policy1(x)
        return self.policy2(x)


class LogMelSpec(nn.Module):

    def __init__(self, sample_rate=8000, n_mels=128, win_length=160, hop_length=80):
        super(LogMelSpec, self).__init__()
        self.transform = torchaudio.transforms.MelSpectrogram(
                            sample_rate=sample_rate, n_mels=n_mels,
                            win_length=win_length, hop_length=hop_length)

    def forward(self, x):
        x = self.transform(x)  # mel spectrogram
        x = np.log(x + 1e-14)  # logrithmic, add small value to avoid inf
        return x


def get_featurizer(sample_rate, n_feats=81):
    return LogMelSpec(sample_rate=sample_rate, n_mels=n_feats,  win_length=160, hop_length=80)


class Data(torch.utils.data.Dataset):

    # this makes it easier to be ovveride in argparse
    parameters = {
        "sample_rate": 8000, "n_feats": 81,
        "specaug_rate": 0.5, "specaug_policy": 3,
        "time_mask": 70, "freq_mask": 15 
    }

    def __init__(self, json_path, sample_rate, n_feats, specaug_rate, specaug_policy,
                time_mask, freq_mask, valid=False, shuffle=True, text_to_int=True, log_ex=True):
        self.log_ex = log_ex
        self.text_process = TextProcess()

        print("Loading data json file from", json_path)
        self.data = pd.read_json(json_path, lines=True)

        if valid:
            self.audio_transforms = torch.nn.Sequential(
                LogMelSpec(sample_rate=sample_rate, n_mels=n_feats,  win_length=160, hop_length=80)
            )
        else:
            self.audio_transforms = torch.nn.Sequential(
                LogMelSpec(sample_rate=sample_rate, n_mels=n_feats,  win_length=160, hop_length=80),
                SpecAugment(specaug_rate, specaug_policy, freq_mask, time_mask)
            )


    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        if torch.is_tensor(idx):
            idx = idx.item()

        try:
            file_path = self.data.key.iloc[idx]
            waveform, _ = torchaudio.load(file_path)
            label = self.text_process.text_to_int_sequence(self.data['text'].iloc[idx])
            spectrogram = self.audio_transforms(waveform) # (channel, feature, time)
            spec_len = spectrogram.shape[-1] // 2
            label_len = len(label)
            if spec_len < label_len:
                raise Exception('spectrogram len is bigger then label len')
            if spectrogram.shape[0] > 1:
                raise Exception('dual channel, skipping audio file %s'%file_path)
            if spectrogram.shape[2] > 1650:
                raise Exception('spectrogram to big. size %s'%spectrogram.shape[2])
            if label_len == 0:
                raise Exception('label len is zero... skipping %s'%file_path)
        except Exception as e:
            if self.log_ex:
                print(str(e), file_path)
            return self.__getitem__(idx - 1 if idx != 0 else idx + 1)  
        return spectrogram, label, spec_len, label_len

    def describe(self):
        return self.data.describe()


def collate_fn_padd(data):
    '''
    Padds batch of variable length

    note: it converts things ToTensor manually here since the ToTensor transform
    assume it takes in images rather than arbitrary tensors.
    '''
    # print(data)
    spectrograms = []
    labels = []
    input_lengths = []
    label_lengths = []
    for (spectrogram, label, input_length, label_length) in data:
        if spectrogram is None:
            continue
       # print(spectrogram.shape)
        spectrograms.append(spectrogram.squeeze(0).transpose(0, 1))
        labels.append(torch.Tensor(label))
        input_lengths.append(input_length)
        label_lengths.append(label_length)

    spectrograms = nn.utils.rnn.pad_sequence(spectrograms, batch_first=True).unsqueeze(1).transpose(2, 3)
    labels = nn.utils.rnn.pad_sequence(labels, batch_first=True)
    input_lengths = input_lengths
    # print(spectrograms.shape)
    label_lengths = label_lengths
    # ## compute mask
    # mask = (batch != 0).cuda(gpu)
    # return batch, lengths, mask
    return spectrograms, labels, input_lengths, label_lengths


## Model

In [22]:
import torch
import torch.nn as nn
from torch.nn import functional as F


class ActDropNormCNN1D(nn.Module):
    def __init__(self, n_feats, dropout, keep_shape=False):
        super(ActDropNormCNN1D, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.norm = nn.LayerNorm(n_feats)
        self.keep_shape = keep_shape
    
    def forward(self, x):
        x = x.transpose(1, 2)
        # x = self.norm(self.dropout(F.gelu(x)))
        x = self.dropout(F.gelu(self.norm(x)))
        if self.keep_shape:
            return x.transpose(1, 2)
        else:
            return x


class SpeechRecognition(nn.Module):
    hyper_parameters = {
        "num_classes": 29,
        "n_feats": 81,
        "dropout": 0.1,
        "hidden_size": 1024,
        "num_layers": 1
    }

    def __init__(self, hidden_size, num_classes, n_feats, num_layers, dropout):
        super(SpeechRecognition, self).__init__()
        self.num_layers = num_layers
        self.hidden_size = hidden_size
        self.cnn = nn.Sequential(
            nn.Conv1d(n_feats, n_feats, 10, 2, padding=10//2),
            ActDropNormCNN1D(n_feats, dropout),
        )
        self.dense = nn.Sequential(
            nn.Linear(n_feats, 128),
            nn.LayerNorm(128),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(128, 128),
            nn.LayerNorm(128),
            nn.GELU(),
            nn.Dropout(dropout),
        )
        self.lstm = nn.LSTM(input_size=128, hidden_size=hidden_size,
                            num_layers=num_layers, dropout=0.0,
                            bidirectional=False)
        self.layer_norm2 = nn.LayerNorm(hidden_size)
        self.dropout2 = nn.Dropout(dropout)
        self.final_fc = nn.Linear(hidden_size, num_classes)

    def _init_hidden(self, batch_size):
        n, hs = self.num_layers, self.hidden_size
        return (torch.zeros(n*1, batch_size, hs),
                torch.zeros(n*1, batch_size, hs))

    def forward(self, x, hidden):
        x = x.squeeze(1)  # batch, feature, time
        x = self.cnn(x) # batch, time, feature
        x = self.dense(x) # batch, time, feature
        x = x.transpose(0, 1) # time, batch, feature
        out, (hn, cn) = self.lstm(x, hidden)
        x = self.dropout2(F.gelu(self.layer_norm2(out)))  # (time, batch, n_class)
        return self.final_fc(x), (hn, cn)

In [26]:
import os
import ast
import torch.optim as optim
from torch.utils.data import DataLoader
from pytorch_lightning.core.lightning import LightningModule
from pytorch_lightning import Trainer
from argparse import ArgumentParser
from pytorch_lightning.loggers import TensorBoardLogger
from pytorch_lightning.callbacks import ModelCheckpoint


class SpeechModule(LightningModule):

    def __init__(self, model, args):
        super(SpeechModule, self).__init__()
        self.model = model
        self.criterion = nn.CTCLoss(blank=28, zero_infinity=True)
        self.args = args

    def forward(self, x, hidden):
        return self.model(x, hidden)

    def configure_optimizers(self):
        self.optimizer = optim.AdamW(self.model.parameters(), self.args.learning_rate)
        self.scheduler = optim.lr_scheduler.ReduceLROnPlateau(
                                        self.optimizer, mode='min',
                                        factor=0.50, patience=6)
        return [self.optimizer], [self.scheduler]

    def step(self, batch):
        spectrograms, labels, input_lengths, label_lengths = batch 
        bs = spectrograms.shape[0]
        hidden = self.model._init_hidden(bs)
        hn, c0 = hidden[0].to(self.device), hidden[1].to(self.device)
        output, _ = self(spectrograms, (hn, c0))
        output = F.log_softmax(output, dim=2)
        loss = self.criterion(output, labels, input_lengths, label_lengths)
        return loss

    def training_step(self, batch, batch_idx):
        loss = self.step(batch)
        logs = {'loss': loss, 'lr': self.optimizer.param_groups[0]['lr'] }
        return {'loss': loss, 'log': logs}

    def train_dataloader(self):
        d_params = Data.parameters
        d_params.update(self.args.dparams_override)
        train_dataset = Data(json_path=self.args.train_file, **d_params)
        return DataLoader(dataset=train_dataset,
                            batch_size=self.args.batch_size,
                            num_workers=self.args.data_workers,
                            pin_memory=True,
                            collate_fn=collate_fn_padd)

    def validation_step(self, batch, batch_idx):
        loss = self.step(batch)
        return {'val_loss': loss}

    def validation_epoch_end(self, outputs):
        avg_loss = torch.stack([x['val_loss'] for x in outputs]).mean()
        self.scheduler.step(avg_loss)
        tensorboard_logs = {'val_loss': avg_loss}
        return {'val_loss': avg_loss, 'log': tensorboard_logs}

    def val_dataloader(self):
        d_params = Data.parameters
        d_params.update(self.args.dparams_override)
        test_dataset = Data(json_path=self.args.valid_file, **d_params, valid=True)
        return DataLoader(dataset=test_dataset,
                            batch_size=self.args.batch_size,
                            num_workers=self.args.data_workers,
                            collate_fn=collate_fn_padd,
                            pin_memory=True)


def checkpoint_callback(args):
    return ModelCheckpoint(
        filepath=args.save_model_path,
        save_top_k=True,
        verbose=True,
        monitor='val_loss',
        mode='min',
        prefix=''
    )

def main(args):
    h_params = SpeechRecognition.hyper_parameters
    h_params.update(args.hparams_override)
    model = SpeechRecognition(**h_params)

    if args.load_model_from:
        speech_module = SpeechModule.load_from_checkpoint(args.load_model_from, model=model, args=args)
    else:
        speech_module = SpeechModule(model, args)

    logger = TensorBoardLogger(args.logdir, name='speech_recognition')
    trainer = Trainer(logger=logger)

    trainer = Trainer(
        max_epochs=args.epochs, gpus=args.gpus,
        num_nodes=args.nodes, distributed_backend=None,
        logger=logger, gradient_clip_val=1.0,
        val_check_interval=args.valid_every,
        checkpoint_callback=checkpoint_callback(args),
        resume_from_checkpoint=args.resume_from_checkpoint
    )
    trainer.fit(speech_module)


# if __name__ == "__main__":
#     parser = ArgumentParser()
#     # distributed training setup
#     parser.add_argument('-n', '--nodes', default=1, type=int, help='number of data loading workers')
#     parser.add_argument('-g', '--gpus', default=1, type=int, help='number of gpus per node')
#     parser.add_argument('-w', '--data_workers', default=0, type=int,
#                         help='n data loading workers, default 0 = main process only')
#     parser.add_argument('-db', '--dist_backend', default='ddp', type=str,
#                         help='which distributed backend to use. defaul ddp')

#     # train and valid
#     parser.add_argument('--train_file', default=None, required=True, type=str,
#                         help='json file to load training data')
#     parser.add_argument('--valid_file', default=None, required=True, type=str,
#                         help='json file to load testing data')
#     parser.add_argument('--valid_every', default=1000, required=False, type=int,
#                         help='valid after every N iteration')

#     # dir and path for models and logs
#     parser.add_argument('--save_model_path', default=None, required=True, type=str,
#                         help='path to save model')
#     parser.add_argument('--load_model_from', default=None, required=False, type=str,
#                         help='path to load a pretrain model to continue training')
#     parser.add_argument('--resume_from_checkpoint', default=None, required=False, type=str,
#                         help='check path to resume from')
#     parser.add_argument('--logdir', default='tb_logs', required=False, type=str,
#                         help='path to save logs')
    
#     # general
#     parser.add_argument('--epochs', default=10, type=int, help='number of total epochs to run')
#     parser.add_argument('--batch_size', default=64, type=int, help='size of batch')
#     parser.add_argument('--learning_rate', default=1e-3, type=float, help='learning rate')
#     parser.add_argument('--pct_start', default=0.3, type=float, help='percentage of growth phase in one cycle')
#     parser.add_argument('--div_factor', default=100, type=int, help='div factor for one cycle')
#     parser.add_argument("--hparams_override", default="{}", type=str, required=False,
# 		help='override the hyper parameters, should be in form of dict. ie. {"attention_layers": 16 }')
#     parser.add_argument("--dparams_override", default="{}", type=str, required=False,
# 		help='override the data parameters, should be in form of dict. ie. {"sample_rate": 8000 }')

#     args = parser.parse_args()
#     args.hparams_override = ast.literal_eval(args.hparams_override)
#     args.dparams_override = ast.literal_eval(args.dparams_override)


#     if args.save_model_path:
#        if not os.path.isdir(os.path.dirname(args.save_model_path)):
#            raise Exception("the directory for path {} does not exist".format(args.save_model_path))

#     main(args)

ImportError: ignored